<a href="https://colab.research.google.com/github/aquapathos/Semi3B/blob/master/noVNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# noVNC + Ngrok

Google Colaboratory はクラウドで動かせる jupyter notebook 環境，基本的には GUI は使えないわけですが，
Ubuntu 18.04 の動作する仮想マシンであり，apt や pip が使え，自由にソフトを追加して使えます．
そこで，まず，

* Xvfb　 仮想ディスプレイ
* X11　   Xサーバと xterm , Web ブラウザ

を導入することで，GUIアプリケーションが動くようにします．


グローバルアドレスが使えるなら，

- VNCサーバ

を立ち上げて，VNC クライアントでリモートデスクトップで使う，ということができるわけですが，
Colaboratory の仮想マシンはプライベートアドレスで動作していますので，外部から接続できません．

## 裏技

VNCクライアントではなく，任意のブラウザでVNCサーバにアクセスできるようにする

- noVNC

というソフトがあります．ブラウザでアクセスということは，これはHTTPのサービスとして稼働させるということです．

一方で，ローカルネットーワ限定でアクセスできるHTTPサーバをポートフォワードを使ってインターネットに公開できるようにしてくれるサービスがいくつかあります．例えば，

- localtunnel
- Ngrok

などです．

noVNC を使って Xvfb 上のデスクトップをブラウザで閲覧できるような HTTPサーバを Colaboratory のマシンのローカルなサーバとして立ち上げ，
それを Ngrok を使ってインターネットに公開してみようというわけです．

HTTPですので，パソコンだけでなく，スマホやiPad などでもアクセス可能です．


In [0]:
%%bash
#VIRTUALGL_VERSION=2.6.1
TURBOVNC_VERSION=2.2.1
LIBJPEG_VERSION=2.0.0
NOVNC_VERSION=1.0.0
WEBSOCKIFY_VERSION=0.8.0
ANYDESK_VERSION=4.0.1-1


CWD=$(pwd)
mkdir -p /opt

apt-get update && apt install -y --no-install-recommends libgtkglext1 libpango1.0-0 libpangox-1.0-0

# TurboVNC + VirtualGL
cd /tmp
curl -fsSL -O https://sourceforge.net/projects/turbovnc/files/${TURBOVNC_VERSION}/turbovnc_${TURBOVNC_VERSION}_amd64.deb
curl -fsSL -O https://sourceforge.net/projects/libjpeg-turbo/files/${LIBJPEG_VERSION}/libjpeg-turbo-official_${LIBJPEG_VERSION}_amd64.deb
#curl -fsSL -O https://sourceforge.net/projects/virtualgl/files/${VIRTUALGL_VERSION}/virtualgl_${VIRTUALGL_VERSION}_amd64.deb
curl -fsSL -O https://download.anydesk.com/linux/anydesk_${ANYDESK_VERSION}_amd64.deb

dpkg -i *.deb
sed -i 's/$host:/unix:/g' /opt/TurboVNC/bin/vncserver
rm -f /tmp/*.deb

# noVNC
curl -fsSL https://github.com/novnc/noVNC/archive/v${NOVNC_VERSION}.tar.gz | tar -xzf - -C /opt 
curl -fsSL https://github.com/novnc/websockify/archive/v${WEBSOCKIFY_VERSION}.tar.gz | tar -xzf - -C /opt 
rm -rf /opt/noVNC
mv /opt/noVNC-${NOVNC_VERSION} /opt/noVNC
rm -rf /opt/websockify
mv /opt/websockify-${WEBSOCKIFY_VERSION} /opt/websockify
ln -s /opt/noVNC/vnc_lite.html /opt/noVNC/index.html
cd /opt/websockify
make  > /dev/null

# X11
apt-get install -y --no-install-recommends \
        ca-certificates \
        curl \
        vim.tiny \
        nano \
        libc6-dev \
        libglu1 \
        libsm6 \
        libxv1 \
        x11-xkb-utils \
        xauth \
        xfonts-base \
        xkb-data \
        openbox \
        xterm \
        scrot \
        xvfb \
        x11vnc \
        xtightvncviewer \
        mesa-utils \
        python-opengl \
    > /dev/null
    
cd ${CWD}

# Xvfb (仮想ディスプレイ)
apt-get -q -y install xvfb > /dev/null

# Web ブラウザ（Epiphany）
add-apt-repository ppa:gnome3-team/gnome3  > /dev/null
apt-get update > /dev/null
apt-get install epiphany-browser > /dev/null


# Ngrok
mkdir -p /content/.vnc
curl -fsSL -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
rm /opt/ngrok
unzip -d /opt ngrok-stable-linux-amd64.zip
rm ngrok-stable-linux-amd64.zip 
echo "web_addr: 4045" > /content/config.yml

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,636 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6



rm: cannot remove '/opt/ngrok': No such file or directory


# VNC 起動

In [0]:
import os
os.environ['PATH'] += "/opt/TurboVNC/bin"

!kill $(ps ax | grep websockify | grep -v grep | awk '{ print $1 }') >/dev/null
!pkill ngrok
!pkill vncserver
!pkill Xvnc
# !pkill Xvfb
!pkill x11vnc

! timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
get_ipython().system_raw('/opt/ngrok http -config=/content/config.yml 5901 &')
! sleep 1
get_ipython().system_raw('DISPLAY=:1 openbox &')
get_ipython().system_raw('DISPLAY=:1 jupyter notebook &')
get_ipython().system_raw('DISPLAY=:1 epiphany http://localhost:8888 &')
get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('DISPLAY=:1 anydesk &')


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


# noVNC転送
表示された URL にアクセスし，表示されているパスワードで仮想デスクトップにアクセス

In [0]:
!grep "one-time password:" /content/.vnc/stdout

import requests
import json
r = requests.get("http://localhost:4045/api/tunnels", headers={"content-type": "application/json"})
print(r.json()['tunnels'][0]['public_url'])

Full control one-time password: 42185720
http://069b561d.ngrok.io
